# 第10章：序列的修改、散列和切片
本章将实现一个多维的扁平Vector类。

## 使用reprlib显示少量元素
reprlib.repr将只显示列表元素的少数元素，大量部分用...替代。  

In [5]:
from array import array
import reprlib

class Vector:
    typecode = "d"

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __repr__(self):
        rst = reprlib.repr(self._components)
        rst = rst[rst.find("["):-1]
        return f"Vector({rst})"

v = Vector(range(10000))
print(v)

Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])


## 协议与不完美的切片
与严格基于接口的语言不同，python可以不通过继承序列类，而仅仅实现序列必须的方法(即len与getitem)来使得对象成为序列。  
在协程的数据protocol中，恐怕也是同样的情况，只是定义的函数不是魔术方法而已。  
> 我个人不喜欢python的这一设计，这提高了灵活性，但往往令人费解。严格使用interface，不但告诉用户应该实现哪些方法，也告诉了用户这些方法的函数签名如何。

> 即使不继承，我们还是不得不实现协议的方法，不是吗？

In [9]:
from array import array
import reprlib

class Vector:
    typecode = "d"

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __repr__(self):
        rst = reprlib.repr(self._components)
        rst = rst[rst.find("["):-1]
        return f"Vector({rst})"

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        return self._components[index]

v = Vector(range(10000))
# Vector实例确实可以切片了，但切片的结果却不是Vector实例，这不符合预期。
print(v[5:10])

array('d', [5.0, 6.0, 7.0, 8.0, 9.0])


## 切片原理与更好的切片
在进行切片或随机访问时，都将调用getitem方法。但getitem方法取得的参数却是不同类型的。

In [2]:
from array import array
import reprlib

class Vector:
    typecode = "d"

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __repr__(self):
        rst = reprlib.repr(self._components)
        rst = rst[rst.find("["):-1]
        return f"Vector({rst})"

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        print(type(index), index)
        print(dir(index))
        return self._components[index]

v = Vector(range(10000))
# 类型是int
# print(v[5])
# 类型是slice
print(v[5:10])


<class 'slice'> slice(5, 10, None)
['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'indices', 'start', 'step', 'stop']
array('d', [5.0, 6.0, 7.0, 8.0, 9.0])


在使用切片访问时，传入的index被构造成一个slice类型。  
切片具有三个要素，start, stop, step。  
slice类型具备关键的indices方法，将缺省的切片要素补全（填充0或length），将-x转换为合理的类型。这非常有用。  
由此，我们可以定义更好的切片，让切片的结果也是一个Vector对象。

In [9]:
from array import array
import reprlib
import numbers

class Vector:
    typecode = "d"
    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __repr__(self):
        rst = reprlib.repr(self._components)
        rst = rst[rst.find("["):-1]
        return f"Vector({rst})"

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        # type在获得某一对象的类，可以用这一方法根据对象构造其他对象。
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            # Vector的getitem访问只应该支持索引与整数切片
            # 而不是表现得像字典。
            msg = f"{cls.__name__} indices must be integers."
            raise TypeError(msg)

v = Vector(range(10000))
print(v[5:10])

Vector([5.0, 6.0, 7.0, 8.0, 9.0])


## 使用setattr与getattr
自定义对象不使用默认的setattr方法时，常可以通过obj.somenewattr来添加本不属于类的属性，很多时候这不符合预期。  
另一种常见情况是，要阻止用户写某些属性。除了使用@property装饰器，还可以通过setattr做得更彻底。  

In [14]:
from array import array
import reprlib
import numbers

class Vector:
    typecode = "d"
    # 希望vector向外提供xyz三个维度的直接访问
    dimattr="xyz"
    def __init__(self, components):
        self._components = array(self.typecode, components)

    # 支持x, y, z访问
    def __getattr__(self, name):
        cls = type(self)
        if len(name)==1:
            pos = cls.dimattr.find(name)
            # 当确实有N个数字在列表中时
            if 0<=pos<len(self._components):
                return self._components[pos]
    
    # 阻止向x, y, z 设值
    def __setattr__(self, name, value):
        if name in self.dimattr and len(name)==1:
            msg = f"read-only attr {name}"
            raise AttributeError(msg)
        # 通过super()访问超类方法
        super().__setattr__(name, value)

    def __repr__(self):
        rst = reprlib.repr(self._components)
        rst = rst[rst.find("["):-1]
        return f"Vector({rst})"

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        # type在获得某一对象的类，可以用这一方法根据对象构造其他对象。
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            # Vector的getitem访问只应该支持索引与整数切片
            # 而不是表现得像字典。
            msg = f"{cls.__name__} indices must be integers."
            raise TypeError(msg)

v = Vector(range(10000))
print(v[5:10])
print(v.x, v.y, v.z)

Vector([5.0, 6.0, 7.0, 8.0, 9.0])
0.0 1.0 2.0


**注意**，如果你为类定义了成员变量，就不能通过setattr来阻止用户修改它。因为你定义时（实际上）也是通过调用setattr来完成的。  
类的编写者和用户具备相同的权力。  
除非是上例的情况，在上例中，xyz实际上并不是Vector的数据成员。只是能通过getattr得到而已。